In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# PART 1: Import Data and Dataframe Creation

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import matplotlib.pyplot as plt

plays_raw_data = pd.read_csv("../input/nfl-big-data-bowl-2022/plays.csv")
plays = plays_raw_data

# type - results
specialTeamsPlay = plays.groupby(['specialTeamsResult', 'specialTeamsPlayType']).size().unstack()
specialTeamsPlay

In [ ]:
# plays_Punt = plays.loc[plays.specialTeamsPlayType=='Punt'].groupby(['specialTeamsResult', 'specialTeamsPlayType']).size().unstack()
# plays_Kickoff = plays.loc[plays.specialTeamsPlayType=='Kickoff'].groupby(['specialTeamsResult', 'specialTeamsPlayType']).size().unstack()
# plays_FieldGoal = plays.loc[plays.specialTeamsPlayType=='Field Goal'].groupby(['specialTeamsResult', 'specialTeamsPlayType']).size().unstack()
# plays_ExtraPoint = plays.loc[plays.specialTeamsPlayType=='Extra Point'].groupby(['specialTeamsResult', 'specialTeamsPlayType']).size().unstack()

# plays_Punt_Result = ['Blocked Punt', 'Downed', 'Fair Catch','Muffed', 'Non-Special Teams Result', 'Out of Bounds', 'Return', 'Touchback']
# plays_Punt

In [ ]:
# plays_Kickoff_Result = ['Downed', 'Fair Catch', 'Kickoff Team Recovery', 'Muffed', 'Out of Bounds', 'Return', 'Touchback']
# plays_Kickoff

In [ ]:
# plays_FieldGoal_Result = ['Blocked Kick Attempt', 'Downed', 'Kick Attempt Good', 'Kick Attempt No Good', 'Non-Special Teams Result', 'Out of Bounds']
# plays_FieldGoal

In [ ]:
# plays_ExtraPoint_Result = ['Blocked Kick Attempt', 'Kick Attempt Good', 'Kick Attempt No Good', 'Non-Special Teams Result']
# plays_ExtraPoint

In [ ]:
# specialTeamsResultList = ['Blocked Kick Attempt', 'Blocked Punt', 'Downed', 
#                           'Fair Catch', 'Kick Attempt Good', 'Kick Attempt No Good', 
#                           'Kickoff Team Recovery', 'Muffed', 'Non-Special Teams Result', 
#                           'Out of Bounds', 'Return', 'Touchback']
# plt.figure(figsize=(8,8))
# plt.pie(plays_ExtraPoint['Extra Point'], explode=(0.5,0.3,0.1,0.5), autopct='%1.1f%%' )
# plt.legend(loc=7,  labels = plays_ExtraPoint_Result, title = "Extra Point:", bbox_to_anchor=(0.6, 0.7))
# plt.show()

# plt.figure(figsize=(8,8))
# plt.pie(plays_FieldGoal['Field Goal'], explode=(0.5, 0.5,0.1,0.1,0.5,0.5), autopct='%1.1f%%')
# plt.legend(loc=7,  labels = plays_FieldGoal_Result, title = "Field Goal:", bbox_to_anchor=(0.85, 0.8))
# plt.show()

# plt.figure(figsize=(8,8))
# plt.pie(plays_Kickoff['Kickoff'], explode=(0.5,0.5,0.5,0.5,0.5,0.1,0.1), autopct='%1.1f%%')
# plt.legend(loc=7,  labels = plays_Kickoff_Result, title = "Kickoff:", bbox_to_anchor=(1, 0.2))
# plt.show()

# plt.figure(figsize=(8,8))
# plt.pie(plays_Punt['Punt'], explode=(0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1), autopct='%1.1f%%' )
# plt.legend(loc=7,  labels = plays_Punt_Result, title = "Punt:", bbox_to_anchor=(1.5, 0.5))
# plt.show()

Create a dataframe that shows the frequency play results with the involvement of each player in kick

In [ ]:
import numpy as np

# player - results
kickers_results = plays.groupby(['kickerId','specialTeamsResult']).size().unstack().replace(np.nan, 0).reset_index()
kickers_results

Create a dataframe that shows the frequency play results with the involvement of each player in return

In [ ]:
returners_results = plays.groupby(['returnerId','specialTeamsResult']).size().unstack().replace(np.nan, 0).reset_index()
returners_results

Read the players dataset and define a function that query players name

In [ ]:
players_raw_data = pd.read_csv("../input/nfl-big-data-bowl-2022/players.csv")
players = players_raw_data

def queryName(i):
    name = players.query('nflId == @i').displayName.values
    return name 

Getting the returners' names

In [ ]:
X=plays
IDList = X.returnerId
NameList = []
for i in IDList.values:
    try:
        if isinstance(i, float):
            NameList.append(np.nan)
        else:
            NameList.append(queryName(int(float(i)))[0])
    except:
        NameList.append(np.nan)

X['returner_name'] = NameList
returnerName_results = X.groupby(['returner_name','specialTeamsResult']).size().unstack().replace(np.nan, 0).reset_index()
returnerName_results

Getting the kickers' names

In [ ]:
IDList = X.kickerId
NameList = []
for i in IDList.values:
    try:
        if isinstance(i, int):
            NameList.append(np.nan)
        else:
            NameList.append(queryName(i)[0])
    except:
        NameList.append(np.nan)
X['kicker_name'] = NameList
kickerName_results = X.groupby(['kicker_name','specialTeamsResult']).size().unstack().replace(np.nan, 0).reset_index()
kickerName_results

Remove duplicates

In [ ]:
# Drop these two rows because of duplicated names:
kickerName_results = kickerName_results[kickerName_results.kicker_name != "Aaron Brewer"]
kickerName_results = kickerName_results[kickerName_results.kicker_name != "Chris Jones"]
kickerName_results = kickerName_results.reset_index(drop=True)
kickerName_results

Get the BMI and position for kicker dataframe

In [ ]:
def queryHeight(i):
    height = players.query('`displayName` == @i').height.values
    return height
def queryWeight(i):
    weight = players.query('`displayName` == @i').weight.values
    return weight
def feetToM(f,i):
    i += f * 12
    return round(i * 2.54, 1)/100
def queryPosition(i):
    position = players.query('`displayName` == @i').Position.values
    return position
def getPositionIndex(i):
    if i == "WR":
        return 0
    elif i == "RB":
        return 1
    elif i == "CB":
        return 2
    elif i == "TE":
        return 3
    elif i == "FB":
        return 4
    elif i == "FS":
        return 5
    elif i == "SS":
        return 6
    elif i == "ILB":
        return 7
    elif i == "OLB":
        return 8
    elif i == "MLB":
        return 9
    elif i == "QB":
        return 10
    elif i == "NT":
        return 11
    elif i == "LB":
        return 12
    elif i == "DE":
        return 13
    elif i == "G":
        return 14
    elif i == "DB":
        return 15
    elif i == "P":
        return 16
    elif i == "K":
        return 17
    else:
        return 20

NameList = kickerName_results.kicker_name
HeightList = []
HeightList2 = []
WeightList = []
BMI = []
PositionList = []

# Obtain H and W 
for i in NameList.values:
    try:
        HeightList.append(queryHeight(i)[0])
        WeightList.append(queryWeight(i)[0])
    except:
        HeightList.append(np.nan)
        WeightList.append(np.nan)

# Change height scale
for i in HeightList:
    try: 
        feet = i.split('-')[0]
        inch = i.split('-')[1]
        m = feetToM(float(feet),float(inch))
        HeightList2.append(m)
    except:
        feet = [char for char in i][0]
        inch = [char for char in i][1]
        m = feetToM(float(feet),float(inch))
        HeightList2.append(m)    

# Compute BMI
for h, w in zip(HeightList2, WeightList):
    BMI.append(round(w * 0.453592 / (h * h), 2))
    
# Obtain Position
for i in NameList.values:
    try:
        PositionList.append(getPositionIndex(queryPosition(i)[0]))
    except:
        PositionList.append(np.nan)
        
kickerName_results2 = kickerName_results.copy(deep=True)
kickerName_results2['BMI'] = BMI
kickerName_results2['Position'] = PositionList
kickerName_results2

Get the BMI and position for returner dataframe

In [ ]:
IDList = X.returnerId
    
NameList = returnerName_results.returner_name
HeightList = []
HeightList2 = []
WeightList = []
BMI = []
PositionList = []

# Obtain H and W 
for i in NameList.values:
    try:
        HeightList.append(queryHeight(i)[0])
        WeightList.append(queryWeight(i)[0])
    except:
        HeightList.append(np.nan)
        WeightList.append(np.nan)

# Change height scale
for i in HeightList:
    try: 
        feet = i.split('-')[0]
        inch = i.split('-')[1]
        m = feetToM(float(feet),float(inch))
        HeightList2.append(m)
    except:
        feet = [char for char in i][0]
        inch = [char for char in i][1]
        m = feetToM(float(feet),float(inch))
        HeightList2.append(m)    

# Compute BMI
for h, w in zip(HeightList2, WeightList):
    BMI.append(round(w * 0.453592 / (h * h), 2))
    
# Obtain Position
for i in NameList.values:
    try:
        PositionList.append(getPositionIndex(queryPosition(i)[0]))
    except:
        PositionList.append(np.nan)
#         print('haha')
      
returnerName_results2 = returnerName_results.copy(deep=True)
returnerName_results2['BMI'] = BMI
returnerName_results2['Position'] = PositionList
returnerName_results2

In [ ]:
# import seaborn as sns
# plt.figure(figsize=(15,30))
# ax = sns.heatmap(kickerName_results,
#                  cmap="PuRd",
#                  vmin=0, vmax=200, annot=True)

# returnerName_results2['Position'].value_counts()
# a = returnerName_results2
# a['Position'].replace({"WR": 0, "b": "y"}, inplace=True)

# PART 2: CLUSTERING - RETURNER

Clustering: find the pattern of returner's information and play results. Preprocessing:

In [ ]:
# clustering preprocessing 
from sklearn.preprocessing import MinMaxScaler

# Instantiate the object
returnerName_results3 = returnerName_results2.drop('returner_name', 1)

scaler = MinMaxScaler()
# Fit and transform the data
# StandardScaler()
returnerName_results3 = scaler.fit_transform(returnerName_results3)
returnerName_results3

Find the optimum number of clusters

In [ ]:
returnerName_results3.shape

In [ ]:
# Import the library
import seaborn as sns
from sklearn.cluster import KMeans
np.random.seed(42)
inertia = []

start = 1
end = 20

# Iterating the process
for i in range(start, end):
  # Instantiate the model
    model = KMeans(n_clusters=i)
  # Fit The Model
    model.fit(returnerName_results3)
  # Extract the error of the model
    inertia.append(model.inertia_)# Visualize the model
sns.pointplot(x=list(range(start, end)), y=inertia)
plt.title('SSE on K-Means based on # of clusters')
plt.show()

clustering

In [ ]:
np.random.seed(42)

# Instantiate the model
results_returner = returnerName_results2.copy(deep=True)
model = KMeans(n_clusters=4)
# Fit the model
model.fit(returnerName_results3)
# Predict the cluster from the data and save it
cluster = model.predict(returnerName_results3)
# Add to the dataframe and show the result
results_returner['cluster'] = cluster
results_returner

visualisation:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create the dataframe to ease our visualization process
visualize = pd.DataFrame(model.cluster_centers_) #.reset_index()
visualize = visualize.T
visualize['column'] = ['Return', 'Fair Catch', 'Muffed', 'Kick Attempt No Good', 'BMI', 'Position']
visualize = visualize.melt(id_vars=['column'], var_name='cluster')
visualize['cluster'] = visualize.cluster.astype('category')
# Visualize the result
plt.figure(figsize=(12, 8))
sns.barplot(x='cluster', y='value', hue='column', data=visualize)
plt.title('The cluster\'s characteristics')
plt.show()

In [ ]:
visualize

# PART 3: CLUSTERING - KICKER

In [ ]:
# Instantiate the object
kickerName_results3 = kickerName_results2.drop('kicker_name', 1)
kickerName_results3 = kickerName_results3.drop('Blocked Punt', 1)
kickerName_results3 = kickerName_results3.drop('Kickoff Team Recovery', 1)
kickerName_results3 = kickerName_results3.drop('Blocked Kick Attempt', 1)
kickerName_results3 = kickerName_results3.drop('Touchback', 1)
kickerName_results3 = kickerName_results3.drop('Downed', 1)
kickerName_results3 = kickerName_results3.drop('Muffed', 1)

column = 'BMI'
kickerName_results3[column] = kickerName_results3[column] /kickerName_results3[column].abs().max()
column = 'Position'
kickerName_results3[column] = kickerName_results3[column] /kickerName_results3[column].abs().max()


scaler = MinMaxScaler()
# Fit and transform the data
# StandardScaler()
kickerName_results3 = scaler.fit_transform(kickerName_results3)



In [ ]:
kickerName_results3.shape

In [ ]:
np.random.seed(42)
inertia = []

start = 1
end = 20

# Iterating the process
for i in range(start, end):
  # Instantiate the model
    model = KMeans(n_clusters=i)
  # Fit The Model
    model.fit(kickerName_results3)
  # Extract the error of the model
    inertia.append(model.inertia_)# Visualize the model
sns.pointplot(x=list(range(start, end)), y=inertia)
plt.title('SSE on K-Means based on # of clusters')
plt.show()

In [ ]:
np.random.seed(42)

# Instantiate the model
results_kicker = kickerName_results2.copy(deep=True)
model = KMeans(n_clusters=3)
# Fit the model
model.fit(kickerName_results3)
# Predict the cluster from the data and save it
cluster = model.predict(kickerName_results3)
# Add to the dataframe and show the result
results_kicker['cluster'] = cluster
results_kicker

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create the dataframe to ease our visualization process
visualize = pd.DataFrame(model.cluster_centers_) #.reset_index()


visualize = visualize.T
# visualize['column'] = ['Downed', 'Fair Catch', 'Muffed', 'Out of Bounds', 'Return', 'Touchback', 'Blocked Kick Attempt', 'Kick Attempt Good', 'Kick Attempt No Good', 'Kickoff Team Recovery', 'Blocked Punt', 'BMI', 'Position']
visualize['column'] = ['Fair Catch', 'Out of Bounds', 'Return', 'Kick Attempt Good', 'Kick Attempt No Good', 'BMI', 'Position']
visualize = visualize.melt(id_vars=['column'], var_name='cluster')
visualize['cluster'] = visualize.cluster.astype('category')
# Visualize the result
plt.figure(figsize=(12, 8))
sns.barplot(x='cluster', y='value', hue='column', data=visualize)
plt.title('The cluster\'s characteristics')
plt.show()